# ECMM423 CA2: Search & Optimisation
An implementation of Differential Evolution (specifically DE/rand/1/bin).  
To use this Notebook, execute all the code blocks in the following sections:  
+ [The Search Algorithm](#search)
+ [Random Search & Stochastic Gradient Descent](#rssgd)
+ [Problem Instances](#prob)  

In the [Parameter Tuning](#param) section, the tuning of the DE parameters for each problem is conducted. Note: execution of the grid search used in parameter tuning takes a long time.  
The [Experiments](#exp) section contains the code for conducting the experiments for each problem on the three algorithms.  
Finally, [Analysis](#ana) contains the results of the experiments with graphs.

<a id='search'></a>
# The Search Algorithm
Run the cells below to define required functions

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from itertools import product
from typing import Callable

In [ ]:
def check_param(NP,f,CR,F,max_gen,d,obj_bounds):
    """
    Checks that the parameters of the differential_evolution()
    function are valid, raising an exception if not.
    """
    # check that the objective function is valid
    if check_obj_func(f, d) == -1:
        raise Exception(f"Objective function must be able to handle {d} dimensional vectors")
    
    if (type(NP) is not int) or (NP<4):
        raise Exception("NP must be an integer greater than or equal to 4.")
    if (type(CR) not in [float, np.float64]) or (CR<0 or CR>1):
        raise Exception("CR must be a float in the range [0,1].")
    if (type(F) not in [float, np.float64]) or (F<=0 or F>2):
        raise Exception("F must be a float in the range (0,2].")
    if (type(max_gen) is not int) or (max_gen<=0):
        raise Exception("max_gen must be a positive integer.")
    if (type(d) is not int) or (d<=0):
        raise Exception("d must be a positive integer.")
    if len(obj_bounds) != 2:
        raise Exception("obj_bounds must be a list-like object of length two.")
    if obj_bounds[0]>=obj_bounds[1]:
        raise Exception("The upper bound must be greater than the lower bound.")
    return 0

In [ ]:
def check_obj_func(f: Callable, d: int):
    x = init_population(1,d)[0]
    try:
        f(x)
    except:
        print(f"Objective function `{f.__name__}` not valid for handling d={d} dimensional vectors.")
        return -1
    else:
        return 0   

In [ ]:
# initialise target vectors:
# initialise a population of size pop_size of random vectors of dimension D
# 
def init_population(NP: int, d: int, lb: float = 0, ub: float = 1):
    """
    Initialise a target population of d-dimensional vectors.
    Defaults for the vector values to be generated in the range [0,1).
    
    Arguments:
    NP -- the size of the population
    d  -- the dimension of each vector
    lb -- lower bound of the vector values
    up -- upper bound of the vector values
    
    Returns:
    an array of length NP of d-dimensional real-valued vectors in the
    range [lb,ub)
    """
    return np.random.rand(NP,d)*(ub-lb)+lb

In [ ]:
def pick_rand_idx(i: int, NP: int):
    """
    Picks three integers from a uniform distribution that are
    different from each other, and from the running index i.
    
    Arguments:
    i -- the running index of the member of the population being mutated
    NP -- the population size to choose integers in the range of
    
    Returns:
    three random integers in the range [0,NP) that are distinct
    and not equal to i
    """
    r1,r2,r3 = np.random.randint(0,NP,3)
    while (r1==i): r1 = np.random.randint(0,NP)
    while (r2==r1) or (r2==i): r2 = np.random.randint(0,NP)
    while (r3==r2) or (r3==r1) or (r3==i): r3 = np.random.randint(0,NP)
    return r1,r2,r3

In [ ]:
# function to return a mutated vector for a particular position in the vector
def mutate_at_pos(x: np.ndarray, F: float, i: int, k: int) -> float:
    """
    Mutates a vector by picking three distinct random vectors and 
    applying a difference mutation at a specific position.
    
    Arguments:
    x -- the current population
    F -- the differential weight
    i -- the current index in the population iteration
    k -- the current index of vector iteration
    
    Returns:
    the mutated value for use at position k in the trial vector"""
    r1,r2,r3 = pick_rand_idx(i,len(x))
    mut = x[r1][k]+F*(x[r2][k]-x[r3][k])
    return mut

In [ ]:
def differential_evolution(NP: int, f: Callable[[np.ndarray],float], CR: float = 0.9, F: float = 0.8,
                           max_gen: int = 100, d: int = 2, obj_bounds: tuple = (-1,1) ):
    """
    Perform the differential evolution algorithm.
    
    Parameters:
    NP -- population size
    f -- the objective function
    CR -- crossover rate
    F  -- differential weight/mutation constant
    max_gen -- maximum number of generations
    d -- number of dimensions of the objective function
    obj_bounds -- bounds of the objective function (defaults in range [-1,1])
                  i.e. the range to search within
    
    Returns:
    the fittest solution after max_gen generations."""
    #check_param(NP,f,CR,F,max_gen,d,obj_bounds) # check all parameters are valid
    
    # initialise arrays
    x = init_population(NP,d,*obj_bounds) # randomised initial population
    u = np.zeros((NP,d)) # array of trial vectors
    x_g = np.zeros((NP,d)) # array for storing generation G+1
    
    n_gen = 0
    while (n_gen<max_gen): # or convergence condition?
        for i in range(NP):
            # generate random numbers for determining mutation/crossover
            rnbr = np.random.randint(d) # random idx 0,...,d-1
            randb = np.random.random(d) # random number in [0,1)
            
            # iterate through each dimension of target vector
            # and mutate its value in that dimension
            for k in range(d): 
                if (k==rnbr) or (randb[k]<CR):
                    u[i][k] = mutate_at_pos(x,F,i,k)
                else:
                    u[i][k] = x[i][k]
            
            # selection: if u[i] fitter than x[i] then use it in next generation
            if f(u[i]) < f(x[i]): x_g[i] = u[i]
            else: x_g[i] = x[i]
        x = x_g
        n_gen+=1
    # determine fitness of every member in population and return the best solution
    fitnesses = np.empty(NP)
    for i, cand in enumerate(x):
        fitnesses[i] = f(cand)
    idx_best = np.argmin(fitnesses)
    return x[idx_best]
        

<a id='rssgd'></a>
# Random Search & Stochastic Gradient Descent
Random search and a stochastic gradient descent implementation. Run cells to define functions.

In [ ]:
def rand_search(NP: int, f: Callable[[np.ndarray],float],max_gen: int = 100, d: int = 2, obj_bounds: tuple = (-1,1)):
    x = init_population(NP,d,*obj_bounds) # randomised initial population
    best_sol = x[0]
    best_score = f(x[0])
    n_gen = 0
    while n_gen < max_gen:
        # initialise new population each time
        x = init_population(NP,d,*obj_bounds)
        for v in x:
            score = f(v)
            if score < best_score:
                best_score = score
                best_sol = v
        n_gen+=1
    return best_sol

In [ ]:
def grad(v, f): #approximate gradient of the function at v
    d = len(v)
    e = 0.0000001 # small e for derivative
    fv = f(v)
    ret_gradient = np.zeros(d)
    loc_gradient = 0
    for i in range(d):
        e_v = np.zeros(d)
        e_v[i] = e
        loc_gradient = (f(v+e_v)-fv)/e
        ret_gradient[i] = loc_gradient
    return ret_gradient

In [ ]:
def grad_descent(NP: int, f: Callable[[np.ndarray],float], learning_rate: float,
                 max_gen: int = 100, d: int = 2, obj_bounds: tuple = (-1,1)):
    
    x = init_population(NP,d,*obj_bounds) # randomised initial population
    n_gen = 0
    while (n_gen<max_gen):
        for v in x: # iterate over vector population
            diff = grad(v, f)*learning_rate
            v -= diff
        n_gen+=1
    
    best_score = None
    best_sol = None
    for v in x:
        score = f(v)
        if best_score is None or score<best_score:
            best_score = score
            best_sol = v
    return best_sol

<a id='prob'></a>
# Problem Instances
Simply execute these blocks to define the problem functions

## Ackley

In [ ]:
# xi in [-32.768, 32.768], minimum f(0,0,...,0) = 0
def ackley(x: np.ndarray):
    a,b,c,d = 20, 0.2, 2*np.pi, len(x)
    ack = -a*np.exp(-b*np.sqrt( 1/d*np.dot(x,x) )) - \
            np.exp( 1/d*np.sum(np.cos(c*x)) ) + a + np.e
    return ack

## Rastrigin

In [ ]:
# xi in [-5.12, 5.12], minimum f(0,0,...,0) = 0
def rastrigin(x: np.ndarray):
    A = 10
    return A*len(x) + np.sum(x*x-A*np.cos(2*np.pi*x))   

## Griewank

In [ ]:
# xi in [-600, 600], minimum f(0,0,...,0) = 0
def griewank(x: np.ndarray):
    return 1+1/4000*np.sum(x*x)-np.product( [np.cos(xi/np.sqrt(i)) for i,xi in enumerate(x,start=1)] )

## Rosenbrock

In [ ]:
# xi in [-5, 10] OR [-2.048, 2.048], minimum f(1,1,...,1) = 0
def rosenbrock(x: np.ndarray):
    d = len(x)
    ros = 0
    for i in range(d-1):
        ros += 100*(x[i+1]-x[i]*x[i])**2+(1-x[i])**2
    return ros

## Easom

In [ ]:
# xi in [-100, 100], 2 Dimensional, minimum f(pi,pi) = -1
def easom(x: np.ndarray):
    x1, x2 = x
    return -(np.cos(x1)*np.cos(x2))*np.exp(-(x1-np.pi)**2-(x2-np.pi)**2)

# Parameter Tuning
In this section a grid search is applied to various problem configurations (F1-F9). The search range for F,CR parameters can be altered in the first code cell. The problem configurations can be altered in the second, along with the allowed generations and total population size. (note: the execution of the main loop can take a long time)

### Tuning DE

In [ ]:
F_range = np.linspace(0.05,1.2,21)
CR_range = np.linspace(0.05,1.0,20)
params_ = {'F': F_range, 'CR': CR_range}

search_space = np.array(list(product(*params_.values())))
print(len(search_space), 'parameter combinations')

In [ ]:
obj_functions = [easom,ackley,ackley,rosenbrock,rosenbrock,rastrigin,rastrigin,griewank,griewank]
dimensions = [2,15,30,15,30,15,30,15,30]
obj_bounds = [(-100,100),(-32.768,32.768),(-32.768,32.768),(-2.048,2.048),(-2.048,2.048),(-5.12,5.12),(-5.12,5.12),(-600,600),(-600,600)]
max_gen = 150
NP = 30

In [ ]:
# RESET DICTIONARIES
best_params = {}
best_score = {}
all_scores = {}

In [ ]:
%%time
for i in range(9):
    obj_f = obj_functions[i]
    d = dimensions[i]
    bounds = obj_bounds[i]
    print(f'Optimising F{i+1} ({obj_f.__name__}, {d}-dim)')
    
    best_params[i] = None
    best_score[i] = None
    all_scores[i] = []
    for params in search_space:
        F_, CR_ = params
        sol = differential_evolution(NP=NP, f=obj_f, F=F_,CR=CR_,
                                  obj_bounds=bounds, max_gen=max_gen, d=d)
        score = obj_f(sol)
        all_scores[i].append(score)
        if best_score[i] is None or score < best_score[i]:
            best_score[i] = score
            best_params[i] = params
    print(f'Best params for F{i+1}: F,CR={best_params[i]} for a score of {best_score[i]}\n')

<a id='exp'></a>
# Experimental Comparison
...with random search and a stochastic hill-climber.  
The first code cell contains the optimum hyperparameters for DE, found from the parameter tuning section. The second code block performs the experiments, and averages the scores at the end.

In [ ]:
de_params = {0: np.array([0.1075, 0.15  ]),
 1: np.array([0.28, 0.75]),
 2: np.array([0.28, 0.95]),
 3: np.array([0.395, 0.85 ]),
 4: np.array([0.28, 0.7 ]),
 5: np.array([0.1075, 0.1   ]),
 6: np.array([0.1075, 0.35  ]),
 7: np.array([0.28, 0.8 ]),
 8: np.array([0.28, 0.95]),}

In [ ]:
obj_functions = [easom,ackley,ackley,rosenbrock,rosenbrock,rastrigin,rastrigin,griewank,griewank]
dimensions = [2,15,30,15,30,15,30,15,30]
obj_bounds = [(-100,100),(-32.768,32.768),(-32.768,32.768),(-2.048,2.048),(-2.048,2.048),(-5.12,5.12),(-5.12,5.12),(-600,600),(-600,600)]
NP = 30
total_iterations = 10
scores = {}

for i in de_params:
    print(i)
    scores[i] = {'de':[],'rs':[],'sgd':[]}
    for n in range(total_iterations):
        F_, CR_ = de_params[i]
        obj_f = obj_functions[i]
        d = dimensions[i]
        bounds = obj_bounds[i]

        de_sol = differential_evolution(NP=NP, f=obj_f, F=F_,CR=CR_,
                                        obj_bounds=bounds, max_gen=333, d=d)
        scores[i]['de'].append( obj_f(de_sol) )
        
        rs_sol = rand_search(NP=NP, f=obj_f, obj_bounds=bounds, max_gen=666, d=d)
        scores[i]['rs'].append( obj_f(rs_sol) )
        
        sgd_sol = grad_descent(NP=NP, f=obj_f, learning_rate=0.005,
                               obj_bounds=bounds, max_gen=333, d=d)
        scores[i]['sgd'].append( obj_f(sgd_sol) )

# calculate the mean of each algorithm's score
avg_scores = {}
for i in scores:
    avg_scores[i] = {}
    avg_scores[i]['de'] = np.mean(scores[i]['de'])
    avg_scores[i]['rs'] = np.mean(scores[i]['rs'])
    avg_scores[i]['sgd'] = np.mean(scores[i]['sgd'])

<a id='ana'></a>
# Analysis of Results

In [ ]:
res = pd.DataFrame(avg_scores).rename({i: f'F{i+1}' for i in range(9)},axis=1).T
print(res.to_string())

As apparent in the table above, DE far outperforms the other two algorithms, converging close to the required values in most cases, and in the cases where it doesn't reach a very optimal solution it still gets a lot closer than the other two methods.